__Author__: Bogdan Bintu

__Email__: bbintu@g.harvard.edu

__Date__:3/4/2020

__Platform__: Python 2.7

In [4]:
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.size']=15
matplotlib.rcParams['font.family']='Arial'

import numpy as np
import os

#### Import corresponding Hi-C data

In [ ]:
import cPickle as pickle
#This data has been downloaded from Rao et al. Cell 2014
hic_fl = r'Data/HiC/si14_500kb_centered_rawReads.pkl'
hic = pickle.load(open(hic_fl,'rb'))

#### Import positions of chromatin loci imaged

In [5]:
folder = r'\Data'

fid = open(folder+os.sep+r'genomic-scale.tsv','r')
lines = np.array([ln[:-1].split('\t')for ln in fid if len(ln)>0])
zxy = np.array(lines[1:,:3][:],dtype=np.float)
fid = open(folder+os.sep+r'genomic-scale-with transcription and nuclear bodies.tsv','r')
lines = np.array([ln[:-1].split('\t')for ln in fid if len(ln)>0])
zxy = np.concatenate([zxy,np.array(lines[1:,:3][:],dtype=np.float)])
zxy = zxy.reshape([-1,2082,3])/1000 #transform to um

#### Calculate distance matrices and contact matrices both cis and trans

In [ ]:
import cPickle as pickle
import matplotlib.pylab as plt
import numpy as np
from scipy.spatial.distance import pdist,cdist,squareform
from tqdm import tqdm_notebook as tqdm

In [ ]:
lens = [76, 80, 66, 63, 60, 55, 53, 48, 40, 43, 44, 44, 33, 30, 31, 30, 33, 33, 33, 33, 31, 31, 51]
ijs = []
for i in range(len(lens)):
    for j in range(len(lens)):
        ijs.append((i,j))
im_med = np.zeros([edges[-1],edges[-1]])
cut_offs = [0.25,0.5,0.75,1]
im_fr = np.zeros([edges[-1],edges[-1],len(cut_offs)])
im_med_trans = []
im_med_cis = []
im_fr_trans = [[] for _ in cut_offs]
im_fr_cis = [[] for _ in cut_offs]

i,j = ijs[1]
for i,j in tqdm(ijs):
    arr = []
    for st1 in [0,edges[-1]]:
        for st2 in [0,edges[-1]]:
            zxy1 = zxy[:,st1+edges[i]:st1+edges[i+1]]
            zxy2 = zxy[:,st2+edges[j]:st2+edges[j+1]]
            arr =arr+[cdist(zxy1[k],zxy2[k]) for k in range(len(zxy1))]
    arr = np.array(arr)
    im_med[edges[i]:edges[i+1],edges[j]:edges[j+1]]=np.nanmedian(arr,axis=0)
    if i==j:
        im_med_cis.append(np.nanmedian(arr[::2],axis=0))
        im_med_trans.append(np.nanmedian(arr[1::2],axis=0))
    for ic,cutoff in enumerate(cut_offs):
        im_fr[edges[i]:edges[i+1],edges[j]:edges[j+1],ic] = 1.*np.sum(arr<cutoff,0)/np.sum(arr>-1,0)
        if i==j:
            im_fr_trans[ic].append(1.*np.sum(arr[1::2]<cutoff,0)/np.sum(arr[1::2]>-1,0))
            im_fr_cis[ic].append(1.*np.sum(arr[::2]<cutoff,0)/np.sum(arr[::2]>-1,0))
pickle.dump([im_med,im_fr,im_med_trans,im_med_cis,im_fr_trans,im_fr_cis,len(zxy)],
        open(r'mat_contact_IMR90_untreated.pkl','wb'))

#### Calculate cis cross-correlation with HiC

In [14]:
#number of regions per chormosome
lens = [76, 80, 66, 63, 60, 55, 53, 48, 40, 43, 44, 44, 33, 30, 31, 30, 33, 33, 33, 33, 31, 31, 51]
edges = np.cumsum([0]+lens)

C:\Users\Bogdan\Anaconda3\envs\py2\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [ ]:
xhic = []
ymed = []
yfr = []
for ichr in range(len(edges)-1):
    xhic.extend(hic[edges[ichr]:edges[ichr+1],edges[ichr]:edges[ichr+1]].ravel())
    ymed.extend(im_med_cis[ichr].ravel())
    im_ = im_fr_cis[1][ichr].copy()
    im_[np.arange(len(im_)),np.arange(len(im_))]=np.nan
    yfr.extend(im_.ravel())

In [ ]:
fig,ax = plt.subplots()
def nan_corr_coef(x_,y_):
    x=np.ravel(x_)
    y=np.ravel(y_)
    keep=(np.isinf(x)==False)&(np.isinf(y)==False)&(np.isnan(x)==False)&(np.isnan(y)==False)
    x=x[keep]
    y=y[keep]
    return np.corrcoef([x,y])[0,1]
rho = nan_corr_coef(xhic,yfr)
plt.loglog(xhic,yfr,'o',color='k',markersize=25,markeredgecolor='k',label='p='+str(np.round(rho,2)),alpha=0.005);
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()
ax.set_xlabel('Contact frequency')
ax.set_ylabel('Hi-C number of contacts')
#plt.axis('equal')
#plt.legend()
#plt.xlim([1,5*10**4])
#plt.ylim([0.1*10**(-2),0.5*10**(0)])